<a href="https://colab.research.google.com/github/kmk4444/LLM/blob/main/Generate_picture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!touch requirements.txt
!echo python-dotenv >> requirements.txt
!echo openai >> requirements.txt
!echo streamlit >> requirements.txt

- **terminal / bash komutu**

In [3]:
pip install -r requirements.txt

In [25]:
%%writefile image_ops.py
from openai import OpenAI
import streamlit as st
import requests # we used that for stability  AI
from io import BytesIO # it provides to move a picture from url to local on html.
import base64 # We added it due to dependency on stability
import os
from dotenv import load_dotenv

#load_dotenv()

#my_key_openai = os.getenv("openai_apikey")
#my_key_stabilityai = os.getenv("stabilityai_apikey")

client = OpenAI(
    api_key="----"
)

my_key_stabilityai="---"

#DALL-E 3 Image Generation

def generate_image(prompt):

    AI_Response = client.images.generate(
        model = "dall-e-3",
        size = "1024x1024",
        quality="hd",
        n=1,
        response_format="url",
        prompt=prompt
    )

    image_url = AI_Response.data[0].url
    revised_prompt = AI_Response.data[0].revised_prompt

    response = requests.get(image_url) # we used get method for requests.
    image_bytes = BytesIO(response.content)

    return image_bytes, revised_prompt



#DALL-E 3 Variation

def create_image_variation(source_image_url):

  AI_Response = client.images.create_variation(
      image=open(source_image_url,"rb"), # we have to send a byte file. rb means read and binary.
      size = "1024x1024",
      n=1, # there are more than one variation but we just decided one variation.
      response_format="url"
  )

  generated_image_url = AI_Response.data[0].url

  response = requests.get(generated_image_url) # it converts url to byte
  image_bytes = BytesIO(response.content) #it saves response like byte format

  return image_bytes



#SD XL Image Generation (Open Source) stable difusion

def generate_with_SD(prompt):
  #there is no SDK so we should determine link.
    url = "https://api.stability.ai/v1/generation/stable-diffusion-xl-1024-v1-0/text-to-image"
  # we create http request because there is no SDK. http consists of url, headers and body.
  # The header carries information about which application we want to access and whether we have authority to access this application.
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {my_key_stabilityai}",
    }

  #body carries data or the main request.
    body = {
        "steps": 40, # how many iterations, it determines quality of picture. when we increase steps, quality will increase.
        "width": 1024,
        "height": 1024,
        "seed": 0,
        "cfg_scale": 5, # determines how faithful it will be to the desired input as text. 0 and 1 provide to increase creativity of the model.
        "samples": 1, # it is similar n or count. how many picture do you want to create? (max:4)
        "text_prompts": [ # we can send more than one prompt.
            {
                "text": prompt,
                "weight": 1 # it means that positive prompt.
            },
            {
                "text": "blurry, bad",
                "weight": -1 # it means that negative prompt.
            }
        ],
    }

    response = requests.post( # we used post method for requests.
        url,
        headers=headers,
        json=body
    )

    data = response.json() # It holds the data contained in the json key.

    return data



tab_generate,tab_variation, tab_SD = st.tabs(["Resim Üret", "Varyasyon Oluştur", "Stable Diffusion"])

with tab_generate:
    st.subheader("DALL-E 3 ile Görsel Oluşturma")
    st.divider()
    prompt = st.text_input("Oluşturmak istediğiniz görseli tarif ediniz")
    generate_btn = st.button("Oluştur")

    if generate_btn:
        image_data, revised_prompt = generate_image(prompt)

        st.image(image=image_data)
        st.divider()
        st.caption(revised_prompt)

with tab_variation:
    st.subheader("DALL-E 3 ile Görsel Varyasyonu Oluşturma")
    st.divider()
    selected_file = st.file_uploader("PNG formatında bir görsel seçiniz", type=["png"])# type determines the format type.

    if selected_file:
        st.image(image=selected_file.name)# it shows our picture which we upload on webpage

    variation_btn = st.button("Varyasyon Oluştur")# we didn't use key because "Oluştur" and "Varyasyon Oluştur" are different.

    if variation_btn:
      image_data= create_image_variation(selected_file.name)

      st.image(image=image_data)

with tab_SD:
  st.subheader("Stable Diffusion ile Görsel Oluşturma")
  st.divider()
  SD_prompt = st.text_input("Oluşturmak istediğiniz görseli tarif ediniz", key="sd_text_input") # we used key because we used two prompt. If don't, it will be mistake.
  SD_generate_btn = st.button("Oluştur", key="sd_button")

  if SD_generate_btn:
    data  = generate_with_SD(SD_prompt)

    #output of generate_with_SD is complex not byte. To analyze output, we write the below code.
    for image in data["artifacts"]: # we loop knowledge of artifacts object
        image_bytes = base64.b64decode(image["base64"]) # it decodes encoder data.
        st.image(image=image_bytes)





Overwriting image_ops.py


In [24]:
!npm install localtunnel
!streamlit run /content/image_ops.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.598s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 2.23s
your url is: https://nine-guests-drop.loca.lt
^C
